In [8]:
import json
import numpy as np
import os
import pandas as pd

from pdathome.constants import global_constants as gc, mappings as mp
from paradigma.windowing import create_segments


l_metrics = ['range_of_motion', 'peak_velocity']
l_aggregates = ['median']
l_quantiles = [0.95]

l_as_measures = ['range_of_motion_median', 'range_of_motion_quantile_95', 'peak_velocity_median', 'peak_velocity_quantile_95']

gd_model_selected = gc.classifiers.RANDOM_FOREST
asd_model_selected = gc.classifiers.LOGISTIC_REGRESSION

l_groupby = [gc.columns.PRE_OR_POST]
l_groupby_side = l_groupby #+ ['affected_side']
l_groupby_segment = l_groupby_side # + [gc.columns.SEGMENT_CAT]
   

def extract_features(df, l_groupby, l_metrics, l_aggregates, l_quantiles=[]):
    # seperate quantiles from other aggregates
    l_df_agg = []
    for metric in l_metrics:
        df_agg = df.groupby(l_groupby)[metric].agg(l_aggregates).reset_index().rename(columns={x: f'{metric}_{x}' for x in l_aggregates})
        df_qs = df.groupby(l_groupby)[metric].quantile(l_quantiles).reset_index()

        for quantile in l_quantiles:
            df_agg[f"{metric}_quantile_{int(quantile*100)}"] = df_qs.loc[df_qs[f'level_{len(l_groupby)}']==quantile, metric].reset_index(drop=True) 

        l_df_agg.append(df_agg)

    for j in range(len(l_df_agg)):
        if j == 0:
            df_agg = l_df_agg[j]
        else:
            df_agg = pd.merge(left=df_agg, right=l_df_agg[j], how='left', on=l_groupby)

    return df_agg


def compute_aggregations(df, l_measures):
    df_agg_side = extract_features(df, l_groupby_side, l_metrics, l_aggregates, l_quantiles)

    d_quant = {}

    for med_stage in df_agg_side[gc.columns.PRE_OR_POST].unique():
        d_quant[med_stage] = {
            'seconds': {
                'overall': df.loc[(df[gc.columns.PRE_OR_POST]==med_stage)].shape[0] / gc.parameters.DOWNSAMPLED_FREQUENCY,
            },
            'values': {}
        }
        for measure in l_measures:
            d_quant[med_stage]['values'][measure] = {
                'overall': df_agg_side.loc[(df_agg_side[gc.columns.PRE_OR_POST]==med_stage), measure].values[0]
            }

    df_agg_side = extract_features(df, l_groupby_segment, l_metrics, l_aggregates, l_quantiles)

    for med_stage in df_agg_side[gc.columns.PRE_OR_POST].unique():
        df_med_stage = df_agg_side.loc[df_agg_side[gc.columns.PRE_OR_POST]==med_stage]
        # for segment_duration_category in df_med_stage[gc.columns.SEGMENT_CAT].unique():
            # df_segment_duration = df_med_stage.loc[df_med_stage[gc.columns.SEGMENT_CAT]==segment_duration_category].copy().reset_index()
            # d_quant[med_stage]['seconds'][segment_duration_category] = df.loc[(df[gc.columns.PRE_OR_POST]==med_stage) & (df[gc.columns.SEGMENT_CAT]==segment_duration_category)].shape[0] / fs
            # for measure in l_measures:
                # d_quant[med_stage]['values'][measure][segment_duration_category] = df_segment_duration[measure].values[0]


        d_quant[med_stage]['seconds']['non_gait'] = d_quant[med_stage]['seconds']['overall']  - np.sum([d_quant[med_stage]['seconds'][segment_duration_category] for segment_duration_category in mp.segment_map.keys() if segment_duration_category in d_quant[med_stage]['seconds'].keys()])

    return d_quant


def compute_effect_size(df, measure, stat):
    df_all_mas = df.loc[df[gc.columns.SIDE]==gc.descriptives.MOST_AFFECTED_SIDE].copy()
    df_all_mas['dataset'] = 'Predicted gait'

    df_pred_mas = df_all_mas.loc[(df_all_mas[gc.columns.PRED_OTHER_ARM_ACTIVITY]==0)].copy()
    df_pred_mas['dataset'] = 'Predicted gait predicted NOAA'

    df_ann_mas = df_all_mas.loc[(df_all_mas['other_arm_activity_boolean']==0)].copy()
    df_ann_mas['dataset'] = 'Predicted gait annotated NOAA'

    df_mas = pd.concat([df_all_mas, df_pred_mas], axis=0).reset_index(drop=True)
    df_mas = pd.concat([df_mas, df_ann_mas], axis=0).reset_index(drop=True)

    # effect size: verschil (point estimate) gedeeld door de std van het verschil (sd van point estimate van bootstrapping)
    d_effect_size = {}
    d_diffs = {}

    for dataset in df_mas['dataset'].unique():
        d_effect_size[dataset] = {}

        for segment_duration in ['overall']:# ['short', 'moderately_long', 'long', 'very_long', 'overall']:
            df_pre = df_mas.loc[(df_mas['dataset']==dataset) & (df_mas[gc.columns.PRE_OR_POST]==gc.descriptives.PRE_MED)]
            df_post = df_mas.loc[(df_mas['dataset']==dataset) & (df_mas[gc.columns.PRE_OR_POST]==gc.descriptives.POST_MED)]

            if segment_duration != 'overall':
                df_pre = df_pre.loc[df_pre[gc.columns.SEGMENT_CAT]==segment_duration]
                df_post = df_post.loc[df_post[gc.columns.SEGMENT_CAT]==segment_duration]

            range_of_motion_pre_vals = df_pre[measure].values
            range_of_motion_post_vals = df_post[measure].values
            
            if len(range_of_motion_pre_vals) != 0 and len(range_of_motion_post_vals) != 0:
                d_effect_size[dataset][segment_duration] = {}
                
                # point estimate (median) of pre-med and post-med for the true sample
                if stat == 'median':
                    d_effect_size[dataset][segment_duration]['mu_pre'] = np.median(range_of_motion_pre_vals)
                    d_effect_size[dataset][segment_duration]['mu_post'] = np.median(range_of_motion_post_vals)
                elif stat == '95':
                    d_effect_size[dataset][segment_duration]['mu_pre'] = np.percentile(range_of_motion_pre_vals, 95)
                    d_effect_size[dataset][segment_duration]['mu_post'] = np.percentile(range_of_motion_post_vals, 95)

                # boostrapping
                bootstrap_pre = np.random.choice(range_of_motion_pre_vals, size=(5000, len(range_of_motion_pre_vals)), replace=True)
                bootstrap_post = np.random.choice(range_of_motion_post_vals, size=(5000, len(range_of_motion_post_vals)), replace=True)

                # point estimate using bootstrapping samples
                if stat == 'median': 
                    bootstrap_samples_pre = np.median(bootstrap_pre, axis=1)
                    bootstrap_samples_post = np.median(bootstrap_post, axis=1)
                elif stat == '95':
                    bootstrap_samples_pre = np.percentile(bootstrap_pre, 95, axis=1)
                    bootstrap_samples_post = np.percentile(bootstrap_post, 95, axis=1)
                    
                # compute difference for std
                bootstrap_samples_diff = bootstrap_samples_post - bootstrap_samples_pre

                # compute the std
                std_bootstrap = np.std(bootstrap_samples_diff)
                d_effect_size[dataset][segment_duration]['std'] = std_bootstrap

                if segment_duration == 'overall':
                    d_diffs[dataset] = bootstrap_samples_diff

    return d_effect_size, d_diffs

l_diffs = []
d_quant = {}
for subject in gc.participant_ids.L_PD_IDS:
    print(f"Processing {subject}...")
    d_quantification = {}
    
    # load gait predictions for free living label
    # df_gait_predictions = pd.read_pickle(os.path.join(PDH_PATH_GAIT_PREDICTIONS, gd_model_selected, f'{subject}.pkl'))
    df_gait_predictions = pd.read_pickle(os.path.join(r'C:\Users\erik_\Documents\PhD\data\pdh_public\preprocessed_data\2.gait_predictions', gd_model_selected, f'{subject}.pkl'))
    df_gait_predictions = df_gait_predictions.loc[df_gait_predictions[gc.columns.SIDE]==gc.descriptives.MOST_AFFECTED_SIDE]

    if subject in gc.participant_ids.L_HC_IDS:
        df_gait_predictions[gc.columns.PRE_OR_POST] = gc.descriptives.CONTROLS

    df_gait_predictions = df_gait_predictions.loc[df_gait_predictions[gc.columns.FREE_LIVING_LABEL]=='Walking', [gc.columns.TIME, gc.columns.PRE_OR_POST]]

    df_gait_predictions[gc.columns.SEGMENT_NR] = create_segments(df=df_gait_predictions, time_column_name=gc.columns.TIME, gap_threshold_s=gc.parameters.SEGMENT_GAP_GAIT)

    # arm activity features
    df_features = pd.read_pickle(os.path.join(r'C:\Users\erik_\Documents\PhD\data\pdh_public\preprocessed_data\3.arm_activity_features', f'{subject}_{gc.descriptives.MOST_AFFECTED_SIDE}.pkl'))

    if subject in gc.participant_ids.L_HC_IDS:
        df_features[gc.columns.PRE_OR_POST] = gc.descriptives.CONTROLS

    df_features['peak_velocity'] = (df_features['forward_peak_ang_vel_mean'] + df_features['backward_peak_ang_vel_mean'])/2
    
    df_ts = pd.read_pickle(os.path.join(r'C:\Users\erik_\Documents\PhD\data\pdh_public\preprocessed_data\3.arm_activity_features', f'{subject}_{gc.descriptives.MOST_AFFECTED_SIDE}_ts.pkl'))

    if subject in gc.participant_ids.L_PD_IDS:
        df_ts_exploded = df_ts.explode([gc.columns.TIME, gc.columns.ARM_LABEL])
    else:
        df_ts_exploded = df_ts.explode([gc.columns.TIME])

    if subject in gc.participant_ids.L_HC_IDS:
        df_ts_exploded[gc.columns.PRE_OR_POST] = gc.descriptives.CONTROLS

    df_features = df_features.drop(columns=[gc.columns.TIME])

    df_features = pd.merge(left=df_features, right=df_ts_exploded, how='right', on=[gc.columns.PRE_OR_POST, gc.columns.SEGMENT_NR, gc.columns.WINDOW_NR])
    df_features = df_features.groupby([gc.columns.TIME, gc.columns.PRE_OR_POST])[['peak_velocity', 'range_of_motion']].mean().reset_index()

    # arm activity predictions
    df_predictions = pd.read_pickle(os.path.join(r'C:\Users\erik_\Documents\PhD\data\pdh_public\preprocessed_data\4.arm_activity_predictions', asd_model_selected, f'{subject}.pkl'))

    # set pred rounded
    df_predictions.loc[df_predictions[gc.columns.PRED_OTHER_ARM_ACTIVITY_PROBA]>=0.5, gc.columns.PRED_OTHER_ARM_ACTIVITY] = 1
    df_predictions.loc[df_predictions[gc.columns.PRED_OTHER_ARM_ACTIVITY_PROBA]<0.5, gc.columns.PRED_OTHER_ARM_ACTIVITY] = 0

    if subject in gc.participant_ids.L_HC_IDS:
        df_predictions[gc.columns.PRE_OR_POST] = gc.descriptives.CONTROLS
    else:
        # boolean for arm activity
        df_predictions.loc[df_predictions[gc.columns.ARM_LABEL]=='Gait without other behaviours or other positions', 'other_arm_activity_boolean'] = 0
        df_predictions.loc[df_predictions[gc.columns.ARM_LABEL]!='Gait without other behaviours or other positions', 'other_arm_activity_boolean'] = 1
        df_predictions.loc[df_predictions[gc.columns.ARM_LABEL]=='Holding an object behind ', gc.columns.ARM_LABEL] = 'Holding an object behind'
        df_predictions[gc.columns.ARM_LABEL] = df_predictions.loc[~df_predictions[gc.columns.ARM_LABEL].isna(), gc.columns.ARM_LABEL].apply(lambda x: mp.arm_labels_rename[x] if x in mp.arm_labels_rename.keys() else x)

    df = pd.merge(left=df_predictions, right=df_gait_predictions, how='left', on=[gc.columns.TIME, gc.columns.PRE_OR_POST])
    df = pd.merge(left=df, right=df_features, how='left', on=[gc.columns.TIME, gc.columns.PRE_OR_POST])

    # PROCESS DATA    

    # unfiltered gait
    d_quantification['unfiltered_gait'] = compute_aggregations(df, l_as_measures)

    # filtered gait
    d_quantification['filtered_gait'] = compute_aggregations(df.loc[df[gc.columns.PRED_OTHER_ARM_ACTIVITY]==0], l_as_measures)

    # no other arm activity (annotated)
    if subject in gc.participant_ids.L_PD_IDS:
        df_diff = pd.DataFrame()
        d_quantification['true_no_other_arm_activity'] = compute_aggregations(df.loc[df['other_arm_activity_boolean']==0], l_as_measures)

        es_mrom, diff_mrom = compute_effect_size(df, 'range_of_motion', 'median')
        es_prom, diff_prom = compute_effect_size(df, 'range_of_motion', '95')

        d_quantification['effect_size'] = {
            'median_rom': es_mrom,
            '95p_rom': es_prom
        }

        for dataset in diff_mrom.keys():
            df_diff = pd.concat([df_diff, pd.DataFrame([subject, dataset, diff_mrom[dataset], diff_prom[dataset]]).T])

        df_diff.columns = [gc.columns.ID, 'dataset', 'median_rom', '95p_rom']

        l_diffs.append(df_diff)

        d_quant[subject] = d_quantification

df_diffs = pd.concat(l_diffs).reset_index(drop=True)

path_output = gc.paths.PATH_OUTPUT

with open(os.path.join(path_output, 'd_quantification.json'), 'w') as f:
    json.dump(d_quant, f)

df_diffs.to_pickle(os.path.join(path_output, 'df_effect_size.pkl'))

Processing hbv002...
Processing hbv012...
Processing hbv014...
Processing hbv015...
Processing hbv016...
Processing hbv017...
Processing hbv022...
Processing hbv024...
Processing hbv039...
Processing hbv043...
Processing hbv047...
Processing hbv054...
Processing hbv065...
Processing hbv077...
Processing hbv079...
Processing hbv090...
Processing hbv013...
Processing hbv018...
Processing hbv023...
Processing hbv038...
Processing hbv058...
Processing hbv063...


In [3]:
df_features = pd.read_pickle(os.path.join(r'C:\Users\erik_\Documents\PhD\data\pdh_public\preprocessed_data\3.arm_activity_features', f'{subject}_{gc.descriptives.MOST_AFFECTED_SIDE}.pkl'))

df_features.head()

,time,pre_or_post,segment_nr,window_nr,other_arm_activity_majority_voting,arm_activity_majority_voting,angle_perc_power,range_of_motion,forward_peak_ang_vel_mean,forward_peak_ang_vel_std,...,cc_3_gyroscope,cc_4_gyroscope,cc_5_gyroscope,cc_6_gyroscope,cc_7_gyroscope,cc_8_gyroscope,cc_9_gyroscope,cc_10_gyroscope,cc_11_gyroscope,cc_12_gyroscope
0,2869.81,pre,1,1,True,non_gait,0.960350,2.790680,4.471501,1.108613,...,18.588606,5.162611,2.236111,2.894101,4.267747,2.038696,-1.331742,2.425525,0.842705,2.557193
1,2910.81,pre,2,1,True,non_gait,0.996568,30.180450,0.000000,0.000000,...,18.436829,6.094022,4.364761,1.692476,5.743548,5.014244,4.923274,4.430027,1.104618,1.824498
2,2911.56,pre,2,2,True,non_gait,0.998259,10.916390,61.101962,0.000000,...,24.453151,8.591188,5.783162,3.657639,-0.987363,4.450801,2.118090,1.922192,2.092054,2.585885
3,2912.31,pre,2,3,True,non_gait,0.998553,16.340827,63.032681,0.000000,...,24.336731,9.190250,3.660480,2.771963,0.171681,4.351659,1.354682,0.434069,2.887293,2.640319
4,2913.06,pre,2,4,True,non_gait,0.999479,16.655821,63.032681,0.000000,...,20.256392,9.554277,5.077694,6.286526,1.801317,2.816584,0.891632,1.728194,3.181987,3.919181
